In [1]:
import pickle
import json
import redshift_connector
import pandas as pd
pd.set_option("display.max_colwidth", None)
import numpy as np

from collections import Counter
from math import ceil
from langdetect import detect

In [2]:
# !pip install redshift-connector
# !pip install pandas
# !pip install matplotlib
# !pip install pyarrow

In [3]:
with open("redshift_creds.txt", "r") as f:
    host = f.readline()[:-1]
    password= f.readline()[:-1]

In [4]:
conn = redshift_connector.connect(
     host=host,
     database='dev',
     user='app_user',
     password=password
  )

In [16]:
# conn.close()

In [6]:
cursor = conn.cursor()

## Notes

### Random Thoughts/Observations

* "mag_normalized_name" seems to be null for all entries in mid.institutions
* will a fine-tuned model be able to pick up on different languages
* should language be an input to the model
* can I use all variations of institution names to augment the training data?
* could use cities/states in the title to map to a country
* will need to have 2 paths: one for when name matches exact to a current normalized institution and one for when there is not an exact match
* need to go back in time or remove a certain % of the institutions from the master institution table in order to simulate the addition of an institution (need the model to be able to choose when to add vs normalize)
* can we use all institutions in a paper to perform entity resolution (in the case there are 3 institutions and country is not available for 1 out of 3, for example)
* can authors be included as well?

### Modeling Process

1. Start with NER to identify part of string that is the institution
2. Get language from string to assist with naming the country (country from string if available and language can be features to the entity resolution model)
3. Take the string and perform some type of entity resolution (using inst. part of string as well as any geo locators)
    * would only need to grab a country in order to distinguish between like named institutions in different countries, hopefully)

### Affiliations to Check with New Model

* 1281735042, 1320721754

#### Institutions

In [135]:
query = """select count(*)
           from mid.institution"""

In [136]:
cursor.execute("ROLLBACK;")
cursor.execute(query)
counts = cursor.fetchall()
print(counts)

([108660],)


In [88]:
query = """select *
           from mid.institution 
           limit 5"""

In [89]:
cursor.execute("ROLLBACK;")
cursor.execute(query)
inst = cursor.fetch_dataframe()
inst.columns

Index(['rank', 'mag_normalized_name', 'display_name', 'official_page',
       'wiki_page', 'paper_count', 'paper_family_count', 'citation_count',
       'iso3166_code', 'latitude', 'longitude', 'created_date',
       'affiliation_id', 'normalized_name', 'grid_id', 'updated_date',
       'ror_id', 'match_name', 'wikidata_id', 'wikipedia_json',
       'wikidata_json', 'merge_into_affiliation_id', 'city', 'region',
       'country', 'geonames_city_id'],
      dtype='object')

In [104]:
query = """select affiliation_id, normalized_name, display_name, match_name, ror_id, city, 
                  region, country, geonames_city_id
           from mid.institution 
           order by RANDOM()
           limit 1000"""

In [105]:
cursor.execute("ROLLBACK;")
cursor.execute(query)
inst = cursor.fetch_dataframe()
inst['match_name'] = inst['match_name'] \
    .str.wrap(30).str.replace("\n", " ")
inst.shape

(1000, 9)

In [107]:
inst.sample(10)

,affiliation_id,normalized_name,display_name,match_name,ror_id,city,region,country,geonames_city_id
887,4210088821,the milton h erickson foundation,The Milton H. Erickson Foundation,themiltonhericksonfoundation,006gcvq32,Phoenix,Arizona,United States,5308655
97,4210123785,australian society for medical research,Australian Society for Medical Research,australiansocietyformedicalres earch,023sa9n23,Sydney,New South Wales,Australia,2147714
342,4210111689,lega italiana protezione uccelli,Lega Italiana Protezione Uccelli,legaitalianaprotezioneuccelli,01prxxc82,Parma,None,Italy,3171457
897,4210147971,arnavutky devlet hastanesi,Arnavutköy Devlet Hastanesi,arnavutkoydevlethastanesi,03wewq802,Istanbul,None,Turkey,745044
71,57502794,federal reserve bank of philadelphia,Federal Reserve Bank of Philadelphia,federalreservebankofphiladelph ia,02gsb6172,Philadelphia,Pennsylvania,United States,4560349
78,4210111729,nia association,Nia Association,niaassociation,01srqm736,Clarksville,Tennessee,United States,4613868
32,4210149705,instituto de medicina tropical pedro kour,Instituto de Medicina Tropical “Pedro Kourí”,institutodemedicinatropical“pe drokouri”,05a9hae73,Havana,None,Cuba,3553478
272,2801384261,north central texas council of governments,North Central Texas Council of Governments,northcentraltexascouncilofgove rnments,05se67h97,Arlington,Texas,United States,4671240
851,4210096048,orkla,Orkla,orkla,00shfde47,Eslöv,None,Sweden,2715946
770,4210101475,centre franois baclesse,Centre François Baclesse,centrefrancoisbaclesse,00mrq0n58,Esch-sur-Alzette,None,Luxembourg,2960596


#### Affiliations

* look into "None" in original_affiliation

In [137]:
query = """select count(*)
           from mid.affiliation"""

In [138]:
cursor.execute("ROLLBACK;")
cursor.execute(query)
counts = cursor.fetchall()
print(counts)

([572385239],)


In [98]:
query = """select *
                 from mid.affiliation
                 limit 5"""

In [99]:
cursor.execute("ROLLBACK;")
cursor.execute(query)
affs = cursor.fetch_dataframe()
affs.columns

Index(['paper_id', 'author_id', 'affiliation_id', 'author_sequence_number',
       'original_author', 'original_affiliation', 'match_author',
       'match_institution_name', 'original_orcid', 'updated_date'],
      dtype='object')

In [117]:
query = """select paper_id, affiliation_id, original_affiliation,
                 match_institution_name
                 from mid.affiliation
                 order by RANDOM()
                 limit 5000"""

In [118]:
cursor.execute("ROLLBACK;")
cursor.execute(query)
affs = cursor.fetch_dataframe()
affs['match_institution_name'] = affs['match_institution_name'] \
    .str.wrap(30).str.replace("\n", " ")
affs.shape

(5000, 4)

In [120]:
affs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   paper_id                5000 non-null   int64  
 1   affiliation_id          2084 non-null   float64
 2   original_affiliation    2441 non-null   object 
 3   match_institution_name  2441 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 156.4+ KB


In [126]:
affs[(affs['affiliation_id'].isnull()) & (affs['original_affiliation'].notnull())].sample(10)

,paper_id,affiliation_id,original_affiliation,match_institution_name
2155,2354872794,NaN,"ENN Research and Development Co,Ltd,Langfang 065001,Hebei Province,China)",ennresearchanddevelopmentcoltd langfang065001hebeiprovincechi na
1827,1503680947,NaN,"1Botamedi Inc Jeju Korea, Republic of",1botamediincjejukorearepublico f
2321,2164871547,NaN,"Christie Hospital, Manchester, United Kingdom",christiehospitalmanchesterunit edkingdom
1106,2126938246,NaN,"Department of Obstetrics and Gynecology, Kaizuka City Hospital, 3-10-20, Hori, Kaiduka, Osaka 597-0015, Japan",departmentofobstetricsandgynec ologykaizukacityhospital31020h orikaidukaosaka5970015japan
1962,2044884715,NaN,"Smur, service urgences, centre hospitalier de Montélimar, Montélimar, France",smurserviceurgencescentrehospi talierdemontelimarmontelimarfr ance
3183,2485061009,NaN,"BioNTech RNA Pharmaceuticals GmbH, Mainz, Germany;",biontechrnapharmaceuticalsgmbh mainzgermany
4319,2801886384,NaN,"Institute of Diagnostic and Interventional Radiology, Raemistrasse, Zurich, Switzerland.",instituteofdiagnosticandinterv entionalradiologyraemistrassez urichswitzerland
4177,3133891541,NaN,"Department of Biochemistry and Genetics, University Hospital Angers, Angers, France.",departmentofbiochemistryandgen eticsuniversityhospitalangersa ngersfrance
733,2972141424,NaN,Санкт-Петербургский государственный институт культуры,санктпетербургскийгосударствен ныйинституткультуры
575,2058243979,NaN,"Rheumatology Department, Mongi Slim Hospital, La Marsa, Tunis, Tunisia.",rheumatologydepartmentmongisli mhospitallamarsatunistunisia


In [121]:
affs.sample(10)

,paper_id,affiliation_id,original_affiliation,match_institution_name
1196,2385187575,NaN,gang,gang
2903,3207489005,62916508.0,"Soil Science, TUM School of Life Sciences Weihenstephan, Technical University of Munich, Germany",soilsciencetumschooloflifescie ncesweihenstephantechnicaluniv ersityofmunichgermany
2034,1999274218,NaN,None,None
1822,2887751214,11957088.0,"Department of Mechanical#N#Engineering-Engineering Mechanics,#N#Michigan Technological University,#N#Houghton, MI 49931#N#e-mail:",departmentofmechanicalnenginee ringengineeringmechanicsnmichi gantechnologicaluniversitynhou ghtonmi49931nemail
2940,1896086024,NaN,None,None
4162,1901899655,NaN,None,None
1036,3096175811,200845125.0,"[School of Computer and Software, Nanjing University of Information Science and Technology, Nanjing, China,]",schoolofcomputerandsoftwarenan jinguniversityofinformationsci enceandtechnologynanjingchina
191,1985392771,103320735.0,"Istituto di Chimica Agraria ed Ambientale, Centro di Ricerche Biotecnologiche, Università Cattolica del Sacro Cuore, Via Milano 24, 26100 Cremona, Italy",istitutodichimicaagrariaedambi entalecentrodiricerchebiotecno logicheuniversitacattolicadels acrocuoreviamilano2426100cremo naitaly
3829,582996075,NaN,None,None
3906,1491291407,NaN,None,None


#### Looking at Specific Paper

In [93]:
query = """select a.paper_id, a.affiliation_id, a.original_affiliation,
                 a.match_institution_name, b.normalized_name, b.display_name, 
                 b.match_name, b.ror_id, b.city, b.region, b.country, b.geonames_city_id
            from(select paper_id, affiliation_id, original_affiliation,
                 match_institution_name
                 from mid.affiliation 
                 where paper_id = 3127589655) a
            left join (select affiliation_id, normalized_name,display_name, 
                       match_name, ror_id, city, region, country, geonames_city_id
                       from mid.institution) b
           on a.affiliation_id=b.affiliation_id"""

In [94]:
cursor.execute("ROLLBACK;")
cursor.execute(query)
test_df = cursor.fetch_dataframe()
test_df.shape
test_df['match_institution_name'] = test_df['match_institution_name'] \
    .str.wrap(30).str.replace("\n", " ")
test_df['match_name'] = test_df['match_name'] \
    .str.wrap(30).str.replace("\n", " ")

In [95]:
test_df.drop_duplicates(subset=['affiliation_id']).head(10)

,paper_id,affiliation_id,original_affiliation,match_institution_name,normalized_name,display_name,match_name,ror_id,city,region,country,geonames_city_id
0,3127589655,1320721754,"Observation and Research Station of the Land Use and Ecological Succession in Huanghuaihai Coal Mining Area, Ministry of Natural Resources, Xuzhou, China",observationandresearchstationo fthelanduseandecologicalsucces sioninhuanghuaihaicoalminingar eaministryofnaturalresourcesxu zhouchina,ministry of natural resources and forestry,Ministry of Natural Resources and Forestry,ministryofnaturalresourcesandf orestry,02ntv3742,Peterborough,Ontario,Canada,6101645
5,3127589655,25757504,"School of Environment and Spatial Informatics, China University of Mining and Technology, Xuzhou, China",schoolofenvironmentandspatiali nformaticschinauniversityofmin ingandtechnologyxuzhouchina,china university of mining and technology,China University of Mining and Technology,chinauniversityofminingandtech nology,01xt2dr21,Xuzhou,None,China,10630003
6,3127589655,1327237609,"Engineering Research Center of Ministry of Education for Mine Ecological Restoration, Xuzhou, China",engineeringresearchcenterofmin istryofeducationformineecologi calrestorationxuzhouchina,ministry of education of the peoples republic of china,Ministry of Education of the People's Republic of China,ministryofeducationofthepeople srepublicofchina,01mv9t934,Beijing,None,China,1816670


#### Joining raw and cleaned data

In [110]:
query = """select a.paper_id, a.affiliation_id, a.original_affiliation,
                 a.match_institution_name, b.normalized_name, b.display_name, 
                 b.match_name, b.ror_id, b.city, b.region, b.country, b.geonames_city_id
            from(select paper_id, affiliation_id, original_affiliation,
                 match_institution_name
                 from mid.affiliation 
                 where affiliation_id is not null
                 order by RANDOM()
                 limit 100000) a
            left join (select affiliation_id, normalized_name,display_name, 
                       match_name, ror_id, city, region, country, geonames_city_id
                       from mid.institution) b
           on a.affiliation_id=b.affiliation_id"""

In [111]:
cursor.execute("ROLLBACK;")
cursor.execute(query)
joined_df = cursor.fetch_dataframe()
joined_df.shape
joined_df['match_institution_name'] = joined_df['match_institution_name'] \
    .str.wrap(30).str.replace("\n", " ")
joined_df['match_name'] = joined_df['match_name'] \
    .str.wrap(30).str.replace("\n", " ")

In [112]:
joined_df.shape

(100000, 12)

In [113]:
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   paper_id                100000 non-null  int64 
 1   affiliation_id          100000 non-null  int64 
 2   original_affiliation    95178 non-null   object
 3   match_institution_name  95178 non-null   object
 4   normalized_name         100000 non-null  object
 5   display_name            100000 non-null  object
 6   match_name              100000 non-null  object
 7   ror_id                  96835 non-null   object
 8   city                    96835 non-null   object
 9   region                  44402 non-null   object
 10  country                 96835 non-null   object
 11  geonames_city_id        96831 non-null   object
dtypes: int64(2), object(10)
memory usage: 9.2+ MB


In [153]:
joined_df.sample(10)

,paper_id,affiliation_id,original_affiliation,match_institution_name,normalized_name,display_name,match_name,ror_id,city,region,country,geonames_city_id
75856,2104299732,168416876,"YUSHI LI, PhD, is an assistant professor of sociology at Northern Kentucky University in Highland Heights. GABRIEL ROGERS is a student and research assistant at the university",yushiliphdisanassistantprofess orofsociologyatnorthernkentuck yuniversityinhighlandheightsga brielrogersisastudentandresear chassistantattheuniversity,northern kentucky university,Northern Kentucky University,northernkentuckyuniversity,01k44g025,Highland Heights,Kentucky,United States,4294788
95768,3191213008,151201029,"Laboratorio de Biomateriales, Biomecánica y Bioinstrumentación, Instituto de Tecnologías Emergentes y Ciencias Aplicadas (ITECA), UNSAM‐CONICET Escuela de Ciencia y Tecnología San Martín Provincia de Buenos Aires Argentina",laboratoriodebiomaterialesbiom ecanicaybioinstrumentacioninst itutodetecnologiasemergentesyc ienciasaplicadasitecaunsam‐con icetescueladecienciaytecnologi asanmartinprovinciadebuenosair esargentina,national scientific and technical research council,National Scientific and Technical Research Council,nationalscientificandtechnical researchcouncil,03cqe8w59,Buenos Aires,None,Argentina,3435910
91412,2029662228,1322918889,"Metabolic Section, Veterans Administration Hospital, Hines, Illinois, and Division of Environmental Studies, Environmental Measurements Laboratory, Department of Energy, New York, New York",metabolicsectionveteransadmini strationhospitalhinesillinoisa nddivisionofenvironmentalstudi esenvironmentalmeasurementslab oratorydepartmentofenergynewyo rknewyork,united states department of veterans affairs,United States Department of Veterans Affairs,unitedstatesdepartmentofvetera nsaffairs,05rsv9s98,Washington D.C.,District of Columbia,United States,4140963
33112,2889879239,76835614,University of Missouri School of Med.,universityofmissourischoolofme d,university of missouri,University of Missouri,universityofmissouri,02ymw8z06,Columbia,Missouri,United States,4381982
81837,2162860019,47720641,"Department of Control Science & Engineering, Huazhong University of Science and Technology, Wuhan, Hubei, 430074, P.R. China",departmentofcontrolscienceengi neeringhuazhonguniversityofsci enceandtechnologywuhanhubei430 074prchina,huazhong university of science and technology,Huazhong University of Science and Technology,huazhonguniversityofscienceand technology,00p991c53,Wuhan,None,China,1791247
93329,1964312519,142476485,Ecole Polytechnique de Tunisie,ecolepolytechniquedetunisie,cole polytechnique,École Polytechnique,ecolepolytechnique,05hy3tk52,Palaiseau,None,France,2988758
28013,1884561001,1281735042,"Natural Resources Canada, Canadian Forest Service - Atlantic Forestry Centre, P.O. Box 4000, Fredericton, New Brunswick, Canada E3B 5P7",naturalresourcescanadacanadian forestserviceatlanticforestryc entrepobox4000frederictonnewbr unswickcanadae3b5p7,natural resources canada,Natural Resources Canada,naturalresourcescanada,05hepy730,Ottawa,Ontario,Canada,6094817
51688,2954928385,200777214,"Second Department of Anesthesiology, Medical School, National and Kapodistrian University of Athens, Athens, Greece -",seconddepartmentofanesthesiolo gymedicalschoolnationalandkapo distrianuniversityofathensathe nsgreece,national and kapodistrian university of athens,National and Kapodistrian University of Athens,nationalandkapodistrianunivers ityofathens,04gnjpq42,Athens,Attiki,Greece,264371
29100,2911589620,2801269068,"Chemistry Branch, Science and Advice for Scottish Agriculture (SASA), Roddinglaw Road, (SASA is a Scientific Division of the Scottish Government Agriculture, Rural and Economy Directorate), Edinburgh, Scotland, UK",chemistrybranchscienceandadvic eforscottishagriculturesasarod dinglawroadsasaisascientificdi visionofthescottishgovernmenta gricultureruralandeconomydirec torateedinburghscotlanduk,scottish government,Scottish Government,scottishgovern

#### Counting possible training data

In [173]:
query = """select count(*) from (select a.paper_id, a.affiliation_id, a.original_affiliation,
                 a.match_institution_name, b.normalized_name, b.display_name, 
                 b.match_name, b.ror_id, b.city, b.region, b.country, b.geonames_city_id
            from(select paper_id, affiliation_id, original_affiliation,
                 match_institution_name
                 from mid.affiliation 
                 where affiliation_id is not null) a
            left join (select affiliation_id, normalized_name,display_name, 
                       match_name, ror_id, city, region, country, geonames_city_id
                       from mid.institution) b
           on a.affiliation_id=b.affiliation_id)
           where original_affiliation LIKE CONCAT('%', CONCAT(display_name, '%'))"""

In [142]:
cursor.execute("ROLLBACK;")
cursor.execute(query)
counts = cursor.fetchall()
print(counts)

([148595579],)


In [174]:
query = """select count(*) from (select a.paper_id, a.affiliation_id, a.original_affiliation,
                 a.match_institution_name, b.normalized_name, b.display_name, 
                 b.match_name, b.ror_id, b.city, b.region, b.country, b.geonames_city_id
            from(select paper_id, affiliation_id, original_affiliation,
                 match_institution_name
                 from mid.affiliation 
                 where affiliation_id is not null) a
            left join (select affiliation_id, normalized_name,display_name, 
                       match_name, ror_id, city, region, country, geonames_city_id
                       from mid.institution) b
           on a.affiliation_id=b.affiliation_id)
           where original_affiliation LIKE CONCAT('%', 
                                                  CONCAT(split_part(display_name,' ',1), 
                                                         CONCAT('%', 
                                                                CONCAT(reverse(split_part(reverse(display_name),' ',1)),'%')))) or
                original_affiliation LIKE CONCAT('%', CONCAT(display_name, '%'))"""

In [175]:
cursor.execute("ROLLBACK;")
cursor.execute(query)
counts = cursor.fetchall()
print(counts)

([167839098],)


In [7]:
query = """select * from (select a.paper_id, a.affiliation_id, a.original_affiliation,
                 a.match_institution_name, b.normalized_name, b.display_name, 
                 b.match_name, b.ror_id, b.city, b.region, b.country, b.geonames_city_id
            from(select paper_id, affiliation_id, original_affiliation,
                 match_institution_name
                 from mid.affiliation 
                 where affiliation_id is not null) a
            left join (select affiliation_id, normalized_name,display_name, 
                       match_name, ror_id, city, region, country, geonames_city_id
                       from mid.institution) b
           on a.affiliation_id=b.affiliation_id)
           where original_affiliation LIKE CONCAT('%', 
                                                  CONCAT(split_part(display_name,' ',1), 
                                                         CONCAT('%', 
                                                                CONCAT(reverse(split_part(reverse(display_name),' ',1)),'%')))) or
                original_affiliation LIKE CONCAT('%', CONCAT(display_name, '%'))
            order by RANDOM() 
            limit 10000"""

In [8]:
cursor.execute("ROLLBACK;")
cursor.execute(query)
training_data = cursor.fetch_dataframe()
training_data.shape
training_data['match_institution_name'] = training_data['match_institution_name'] \
    .str.wrap(30).str.replace("\n", " ")
training_data['match_name'] = training_data['match_name'] \
    .str.wrap(30).str.replace("\n", " ")

In [11]:
training_data['display_name_ind'] = training_data.apply(lambda x: 1 if x.display_name in x.original_affiliation else 0, axis=1)

In [14]:
training_data[training_data['display_name_ind']==0].shape

(1163, 13)

In [15]:
training_data.shape

(10000, 13)